In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

from langchain.prompts import load_prompt

from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [5]:
# 디스크에서 prompt template를 가져오는 방법

In [6]:
# json 로드
prompt = load_prompt("../prompt.json")
prompt.format(country="Germany")

'What is the capital of Germany'

In [11]:
# yaml 로드
prompt = load_prompt("../prompt.yaml")
prompt.format(country="Korea")

'What is the capital of Korea'

In [15]:
# Serialize(직렬화)는 불러오기 저장 등을 의미
# Compose는 다양한 작은 prompt template를 가지고 이들을 모두 결합하는 것을 의미

# 많은 prompt들의 memory 등을 다 모으는 방법
# 대량의 prompt들이 여러 개로 나눠져 있을때

# PipelinePromptTemplate는 많은 prompt들을 하나로 합칠 수 있게 해줌

In [16]:
# 4개의 prompt
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
    {example}
    {start}
"""
)

In [29]:
pipeline_prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    # 여러 개의 prompt를 pipeline으로 연결해서 만드는 거
    final_prompt=final,  # 최종 출력되는 prompt
    pipeline_prompts=pipeline_prompts,  # prompt pipeline
)

full_prompt.format(
    character="Pirate",
    example_question="What is your location?",
    example_answer="Arrrrg! That is a secret!! Arg arg!!",
    question="What is your fav food?",
)

'\n    \n    You are a role playing assistant.\n    And you are impersonating a Pirate\n\n    \n    This is an example of how you talk:\n\n    Human: What is your location?\n    You: Arrrrg! That is a secret!! Arg arg!!\n\n    \n    Start now!\n\n    Human: What is your fav food?\n    You:\n\n'

In [28]:
chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrr matey! Me favorite food be a hearty stew made with plenty of fresh fish and vegetables, seasoned with me secret blend of spices. It be a meal fit for a pirate king! Arrr!

AIMessageChunk(content='Arrr matey! Me favorite food be a hearty stew made with plenty of fresh fish and vegetables, seasoned with me secret blend of spices. It be a meal fit for a pirate king! Arrr!')